In [1]:
import argparse
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto", # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [3]:
# Load the databricks dataset from Hugging Face
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

Extracting data files: 100%|██████████████████████████████████████████████| 1/1 [00:00<00:00, 892.98it/s]
Generating train split: 15011 examples [00:00, 652334.30 examples/s]


In [6]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

# *** OUTPUT ***
# Number of prompts: 15011
# Column Names are: ['instruction', 'context', 'response', 'category']

Number of prompts: 15011
Column names are: ['instruction', 'context', 'response', 'category']


In [7]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction', 'context', 'response')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """

    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruction:"
    INPUT_KEY = "Input:"
    RESPONSE_KEY = "### Response:"
    END_KEY = "### End"
    
    blurb = f"{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}\n{sample['instruction']}"
    input_context = f"{INPUT_KEY}\n{sample['context']}" if sample["context"] else None
    response = f"{RESPONSE_KEY}\n{sample['response']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    
    sample["text"] = formatted_prompt

    return sample

In [8]:
# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)
    
    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["instruction", "context", "response", "text", "category"],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [9]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [10]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=16,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config

In [11]:
# SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [12]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

In [14]:
# Load model from HF with user's token and with bitsandbytes config
model_name = "/mnt/repo/text-generation-webui/models/meta-llama_Llama-2-7b-chat-hf"
# model_name = "meta-llama/Llama-2-7b-hf" 

bnb_config = create_bnb_config()

model, tokenizer = load_model(model_name, bnb_config)

Loading checkpoint shards: 100%|███████████████████████████████████████████| 2/2 [00:02<00:00,  1.22s/it]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [17]:
## Preprocess dataset

max_length = get_max_length(model)

dataset = preprocess_dataset(tokenizer, max_length, 1, dataset)

Found max lenth: 4096
Preprocessing dataset...


Filter: 100%|█████████████████████████████████████████████| 15011/15011 [00:01<00:00, 8279.64 examples/s]


In [18]:
def train(model, tokenizer, dataset, output_dir):
    # Apply preprocessing to the model to prepare it by
    # 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # 2 - Using the prepare_model_for_kbit_training method from PEFT
    model = prepare_model_for_kbit_training(model)

    # Get lora module names
    modules = find_all_linear_names(model)

    # Create PEFT config for these modules and wrap the model to PEFT
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    
    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)
    
    # Training parameters
    trainer = Trainer(
        model=model,
        train_dataset=dataset,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            max_steps=20,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=1,
            output_dir="outputs",
            optim="paged_adamw_8bit",
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
    )
    
    model.config.use_cache = False  # re-enable for inference to speed up predictions for similar inputs
    
    ### SOURCE https://github.com/artidoro/qlora/blob/main/qlora.py
    # Verifying the datatypes before training
    
    dtypes = {}
    for _, p in model.named_parameters():
        dtype = p.dtype
        if dtype not in dtypes: dtypes[dtype] = 0
        dtypes[dtype] += p.numel()
    total = 0
    for k, v in dtypes.items(): total+= v
    for k, v in dtypes.items():
        print(k, v, v/total)
     
    do_train = True
    
    # Launch training
    print("Training...")
    
    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)    
    
    ###
    
    # Saving model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok=True)
    trainer.model.save_pretrained(output_dir)
    
    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()
    
    
output_dir = "results/llama2/final_checkpoint"
train(model, tokenizer, dataset, output_dir)

[codecarbon INFO @ 03:51:32] [setup] RAM Tracking...
[codecarbon INFO @ 03:51:32] [setup] GPU Tracking...
[codecarbon INFO @ 03:51:32] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 03:51:32] [setup] CPU Tracking...
[codecarbon INFO @ 03:51:32] Tracking Intel CPU via RAPL interface


all params: 3,540,389,888 || trainable params: 39,976,960 || trainable%: 1.1291682911958425


[codecarbon INFO @ 03:51:33] >>> Tracker's metadata:
[codecarbon INFO @ 03:51:33]   Platform system: Linux-5.19.0-1010-nvidia-lowlatency-x86_64-with-glibc2.35
[codecarbon INFO @ 03:51:33]   Python version: 3.10.12
[codecarbon INFO @ 03:51:33]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 03:51:33]   Available RAM : 125.687 GB
[codecarbon INFO @ 03:51:33]   CPU count: 16
[codecarbon INFO @ 03:51:33]   CPU model: 11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz
[codecarbon INFO @ 03:51:33]   GPU count: 1
[codecarbon INFO @ 03:51:33]   GPU model: 1 x NVIDIA GeForce RTX 3090
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.float32 302387200 0.08541070604255438
torch.uint8 3238002688 0.9145892939574456
Training...


Step,Training Loss
1,2.581400
2,2.256600
3,2.248600
4,1.955800
5,1.597200
6,1.602200
7,1.410800
8,1.404100
9,1.489500
10,1.262200


[codecarbon INFO @ 03:51:51] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.13263654708862 W
[codecarbon INFO @ 03:51:51] Energy consumed for all GPUs : 0.001534 kWh. Total GPU Power : 368.11400000000003 W
[codecarbon INFO @ 03:51:51] Energy consumed for all CPUs : 0.000196 kWh. Total CPU Power : 46.99602307540142 W
[codecarbon INFO @ 03:51:51] 0.001926 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:52:06] Energy consumed for RAM : 0.000393 kWh. RAM Power : 47.13263654708862 W
[codecarbon INFO @ 03:52:06] Energy consumed for all GPUs : 0.003061 kWh. Total GPU Power : 366.531 W
[codecarbon INFO @ 03:52:06] Energy consumed for all CPUs : 0.000391 kWh. Total CPU Power : 46.856872573325454 W
[codecarbon INFO @ 03:52:06] 0.003845 kWh of electricity used since the beginning.
[codecarbon INFO @ 03:52:21] Energy consumed for RAM : 0.000589 kWh. RAM Power : 47.13263654708862 W
[codecarbon INFO @ 03:52:21] Energy consumed for all GPUs : 0.004595 kWh. Total GPU Power :

***** train metrics *****
  epoch                    =       0.01
  total_flos               =   357454GF
  train_loss               =     1.5302
  train_runtime            = 0:00:46.46
  train_samples_per_second =      1.722
  train_steps_per_second   =       0.43
{'train_runtime': 46.4666, 'train_samples_per_second': 1.722, 'train_steps_per_second': 0.43, 'total_flos': 383814189195264.0, 'train_loss': 1.5302437514066696, 'epoch': 0.01}
Saving last checkpoint of the model...
